In [99]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np

In [100]:
url = "http://www.boattrader.com/search-results/NewOrUsed-any/Type-any/Category-all/Zip-14853/Radius-200/Sort-Length:DESC"
main_url = "http://www.boattrader.com" #/listing/2006-sea-ray-320-sundancer-103069603/

def string_check(value):
    if value is None:
        return "" 
    return str(value.text)

def from_table(soup,name):
    
    data = list()
    #columns = ['Class','Category']#'Year','Make','Length','Propulsion','Type','Hull Material','Fuel Type','Location']
    if(soup.find('div',{'id':name}) is not None):
        table = soup.find('div',{'id':name}).find('table')
        rows = table.find_all('tr')
        for row in rows:
            if(name =='measurements'):
                index = string_check(row.find('dt'))
                val  = string_check(row.find('dd')) 
            else:
                index = string_check(row.find('th'))
                val  = string_check(row.find('td'))
            
            #if(index in columns):
            data.append((index,val))
            
    return data
        

def getsoup(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r,'html.parser')
    return soup

def check_element(element):
    return element if element is not None else " "


def scrape_page(s):
    
    a = s.find("h1",attrs={"class":"bd-name"})
    b = s.find("div",attrs={"class":"bd-price-location"})
    data = []
    df = pd.DataFrame()
    if a is not None:
        year =  str(check_element(a.find('span',{"class":"bd-year"})).get_text())
        make =  str(check_element(a.find('span',{"class":"bd-make"})).get_text())
        model = str(check_element(a.find('span',{"class":"bd-model"})).get_text())
        data.append(("year-primary",year))
        data.append(("make-primary",make))
        data.append(("model",model))
        
    if b is not None:                
        price = str(check_element(b.find('span',{"class":"bd-price contact-toggle"})).get_text()).split()[0]
        data.append(("price",price))
    #else:
    #    data.append(" ")
    content = s.find('div',{'class':'detail-tbl'})
    if content is not None:
        data  = data + from_table(content,'boat-details')
        #data  = data + from_table(content,'measurements')
        #data  = data + from_table(content,'propulsion')
        data  = data + from_table(content,'other-details')
        #data = pd.DataFrame.from_records(data)
        list1,list2 = zip(*data)
        df  = pd.DataFrame(list(list2),index = (list(list1))).transpose()
    print(df)
    return df

def scrape_full_page(url):
    soup = getsoup(url)
    a = soup.find_all("a", attrs={"data-reporting-click-listing-type":"standard listing"})
    links = [str(main_url+x['href']) for x in a]
    data = pd.DataFrame()

    for link in links:
        s = getsoup(link)
        data = pd.concat([data,scrape_page(s)], axis=0, ignore_index=True) 
    return data
    
    
#     details = s.find('section',{"id":"collapsible-content-areas"}) 
#     details = s.find('section',{"id":"collapsible-content-areas"}) 
#     from_table(details,'boat-details')


In [103]:
df = scrape_full_page(url)
soup = getsoup(url)
link = soup.find('a',{'class':'next'})
while link is not None:
    url = str(main_url+link['href'])
    df = pd.concat([df,scrape_full_page(url)], axis=0, ignore_index=True) 
    link = soup.find('a',{'class':'next'})
df.to_csv("Boat.csv")
#columns = ['Year','Make','Model','Price','Class','Category','Year1','Make1','Length','Propulsion','Type','Hull Material','Fuel Type','Location']
#t = pd.DataFrame(df,columns = columns)


  year-primary make-primary          model    price  Class   Category  Year  \
0         2006      Sea Ray  320 Sundancer  $99,995  Power   Cruisers  2006   

      Make Length Propulsion Type Hull Material Fuel Type         Location  \
0  Sea Ray   320'           Other         Other     Other  N Tonawanda, NY   

  Hull Shape  
0      Other  
  year-primary    make-primary  model    price  Class     Category  Year  \
0         2012  Stingray Boats  215LR  $27,900  Power   Deck Boats  2012   

             Make Length Propulsion Type Hull Material Fuel Type    Location  \
0  Stingray Boats   263'           Other         Other     Other  Tafton, PA   

  Hull Shape  
0      Other  
  year-primary       make-primary  model    price  Class    Category  Year  \
0         2016  Yamaha Sport Boat  SX240  $45,999  Power   Jet Boats  2016   

                Make Length Propulsion Type Hull Material Fuel Type  \
0  Yamaha Sport Boat   240'           Other    Fiberglass             

          

  year-primary make-primary            model       price  Class  \
0         2016     Princess  S65 Sportbridge  $2,795,000  Power   

                     Category  Year      Make Length Propulsion Type  \
0   Flybridge, Sports Cruiser  2016  Princess    65'           Other   

  Hull Material Fuel Type       Location  
0    Fiberglass    Diesel  Glen Cove, NY  
  year-primary make-primary           model     price  Class  \
0         2003       Viking  65 Convertible  $995,000  Power   

                                   Category  Year    Make Length  \
0   Convertible Boats, Sports Fishing Boats  2003  Viking    65'   

  Propulsion Type Hull Material Fuel Type       Location  
0           Other    Fiberglass    Diesel  Staten Island  
  year-primary     make-primary  model    price  Class  \
0         2018  Prestige Yachts  630 S  Request  Power   

                         Category  Year             Make Length  \
0   Express Cruiser, Motor Yachts  2018  Prestige Yachts    63'   

  year-primary make-primary         model    price  Class  \
0         2016    Sunseeker  Manhattan 55  Request  Power   

                   Category  Year       Make Length Propulsion Type  \
0   Flybridge, Motor Yachts  2016  Sunseeker    55'    Twin Inboard   

  Hull Material Fuel Type             Location  
0    Fiberglass    Diesel  Port Washington, NY  
  year-primary make-primary            model       price  Class  \
0         2016     Princess  S65 Sportbridge  $2,795,000  Power   

                     Category  Year      Make Length Propulsion Type  \
0   Flybridge, Sports Cruiser  2016  Princess    65'           Other   

  Hull Material Fuel Type       Location  
0    Fiberglass    Diesel  Glen Cove, NY  
  year-primary make-primary           model     price  Class  \
0         2003       Viking  65 Convertible  $995,000  Power   

                                   Category  Year    Make Length  \
0   Convertible Boats, Sports Fishing Boats  2003  Viking    65'   

  Pr

  year-primary  make-primary        model     price  Class  \
0         2001  Ocean Yachts  Super Sport  $339,000  Power   

                                   Category  Year          Make Length  \
0   Sports Fishing Boats, Convertible Boats  2001  Ocean Yachts    56'   

  Propulsion Type Hull Material Fuel Type             Location    Hull Shape  
0    Twin Inboard    Fiberglass    Diesel  Laurence Harbor, NJ  Modified Vee  
  year-primary make-primary         model    price  Class  \
0         2016    Sunseeker  Manhattan 55  Request  Power   

                   Category  Year       Make Length Propulsion Type  \
0   Flybridge, Motor Yachts  2016  Sunseeker    55'    Twin Inboard   

  Hull Material Fuel Type             Location  
0    Fiberglass    Diesel  Port Washington, NY  
  year-primary make-primary    model     price  Class   Category  Year  \
0         2011      Uniesse  Cruiser  $899,000  Power   Cruisers  2011   

      Make Length Propulsion Type Hull Material Fuel Ty

  year-primary make-primary           model    price  Class   Category  Year  \
0         1992     Jeanneau  Sun Odyssey 51  $95,000  Sails   Cruisers  1992   

       Make Length Propulsion Type Hull Material Fuel Type         Location  
0  Jeanneau    51'           Other    Fiberglass    Diesel  Sodus Point, NY  
  year-primary make-primary                           model     price  Class  \
0         2005      Sea Ray  460 Sundancer - Brokerage Boat  $269,000  Power   

        Category  Year     Make Length Propulsion Type Hull Material  \
0   Motor Yachts  2005  Sea Ray    51'           Other    Fiberglass   

  Fuel Type            Location Hull Shape  
0     Other  Alexandria Bay, NY      Other  
  year-primary make-primary          model     price  Class          Category  \
0         2001      Sea Ray  510 Sundancer  $249,000  Power   Express Cruiser   

   Year     Make Length Propulsion Type Hull Material Fuel Type  \
0  2001  Sea Ray    51'    Twin Inboard    Fiberglass    

...

  year-primary make-primary      model    price  Class    Category  Year  \
0         2017         Lowe  Jon L1032  Request  Power   Jon Boats  2017   

   Make Length Propulsion Type Hull Material Fuel Type        Location  \
0  Lowe    10'           Other      Aluminum            Saugerties, NY   

     Hull Shape Hull Warranty  
0  Modified Vee       99 year  
  year-primary make-primary      model    price  Class    Category  Year  \
0         2017         Lowe  Jon L1040  Request  Power   Jon Boats  2017   

   Make Length Propulsion Type Hull Material Fuel Type        Location  \
0  Lowe    10'           Other      Aluminum            Saugerties, NY   

     Hull Shape Hull Warranty  
0  Modified Vee       99 year  
  year-primary make-primary      model    price  Class    Category  Year  \
0         2017         Lowe  Jon L1032  Request  Power   Jon Boats  2017   

   Make Length Propulsion Type Hull Material Fuel Type        Location  \
0  Lowe    10'           Other      Alumi

  year-primary make-primary                         model price  Class  \
0         2014      Tracker  Topper 1032 Riveted Jon Boat  $539  Power   

                                            Category  Year     Make Length  \
0   Utility Boats, Aluminum Fish Boats, Freshwate...  2014  Tracker    10'   

  Propulsion Type Hull Material Fuel Type   Location  \
0           Other      Aluminum            Utica, NY   

                                     Pricing Details           HIN Hull Shape  \
0  \n\nBase Price: $499\nFreight: $40\nYour Price...  BUJ24235D414       Flat   

  Hull Warranty  
0        3 year  
  year-primary make-primary     model    price  Class   Category  Year  \
0         1983       Trojan  10 Meter  $22,000  Power   Cruisers  1983   

     Make Length Propulsion Type Hull Material Fuel Type           Location  
0  Trojan    10'    Twin Inboard    Fiberglass       Gas  Union Springs, NY  
  year-primary       make-primary      model    price Class  \
0         2017

  year-primary make-primary                    model price  Class    Category  \
0         2018      Tracker  Topper 1036 Riveted Jon  $629  Power   Jon Boats   

   Year     Make Length  Propulsion Type Hull Material Fuel Type  \
0  2018  Tracker    10'  Single Outboard      Aluminum             

         Location                                    Pricing Details  \
0  Harrisburg, PA  \n\nBase Price: $579\nFreight: $50\nYour Price...   

            HIN Hull Shape  
0  BUJ44735F718       Flat  
  year-primary make-primary                    model    price  Class  \
0         2017      Tracker  Topper 1036 Riveted Jon  Request  Power   

         Category  Year     Make Length  Propulsion Type Hull Material  \
0   Utility Boats  2017  Tracker    10'  Single Outboard      Aluminum   

  Fuel Type    Location           HIN  
0            Auburn, NY  BUJ20051D717  
  year-primary make-primary                    model price  Class  \
0         2017      Tracker  Topper 1036 Riveted Jon  

  year-primary make-primary                        model    price Class  \
0         2017      Sea-Doo  SPARK 3-up Rotax 900 HO ACE  Request   PWC   

               Category  Year     Make Length Propulsion Type Hull Material  \
0   Personal Watercraft  2017  Sea-Doo     9'  Single Inboard    Fiberglass   

  Fuel Type          Location  
0       Gas  Grand Island, NY  
  year-primary make-primary        model   price Class              Category  \
0         2017      Sea-Doo  Spark Trixx  $7,299   PWC   Personal Watercraft   

   Year     Make Length Propulsion Type Hull Material Fuel Type    Location  
0  2017  Sea-Doo     9'           Other         Other       Gas  Cicero, NY  
  year-primary make-primary                                        model  \
0         2014      Sea-Doo  Spark 2up ROTAX 900 ACE Convenience Package   

    price Class              Category  Year     Make Length Propulsion Type  \
0  $9,495   PWC   Personal Watercraft  2014  Sea-Doo     9'  Single Inboard  

KeyError: 'href'